In [2]:
# pip install ipython-sql use it to install sql magic to jupyter.....

import sqlite3 as sql
import pandas as pd

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");


 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

In [5]:
%%sql

WITH
    invoice_lines_genres AS
    (
        SELECT 
            il.invoice_id,
            g.name as genre
        FROM genre g
        INNER JOIN track t ON t.genre_id = g.genre_id
        INNER JOIN invoice_line il ON t.track_id = il.track_id
    ),
    invoice_genre AS
    (
        SELECT 
            i.invoice_id,
            i.billing_country,
            ilg.genre
        FROM invoice i
        INNER JOIN invoice_lines_genres ilg ON ilg.invoice_id = i.invoice_id
        WHERE billing_country = "USA"
    )
    
    
SELECT genre,
       COUNT(invoice_id) tracks_sold,
        CAST(COUNT(invoice_id) AS FLOAT)  / CAST((SELECT COUNT(invoice_id) FROM invoice_genre) AS FLOAT) percentage_sold
        FROM invoice_genre
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10




 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


As we can see, Rock music was the most succesfull genre with over 50% of sales in the US!.
From here we could say that Blues Punk and Pop can be introduced in the store.

## Analyzing Employee Sales Performance

In [6]:
%%sql

WITH
    employee_invoicing AS
    (
        SELECT 
            e.first_name||" "||e.last_name AS employee_name,
            e.title AS position,
            e.*,
            c.first_name||" "||c.last_name AS customer_name,
            c.customer_id
        FROM employee AS e
        INNER JOIN customer c ON c.support_rep_id = e.employee_id
        WHERE position = "Sales Support Agent"
    )
    
    

SELECT 
    ei.employee_name,
    SUM(i.total) Total_Sales,
    ei.hire_date
    FROM employee_invoicing ei
    INNER JOIN invoice i ON i.customer_id = ei.customer_id
    GROUP BY 1
    ORDER BY 1


 * sqlite:///chinook.db
Done.


employee_name,Total_Sales,hire_date
Jane Peacock,1731.5100000000039,2017-04-01 00:00:00
Margaret Park,1584.0000000000034,2017-05-03 00:00:00
Steve Johnson,1393.920000000002,2017-10-17 00:00:00


Here we see that Jane is way ahead other sales agents with about 2000 dollars more. Steve is in last place. One explanation could be the experience that each of them has in the company. For example Jane, was hired 6 months before Steve and one month before Margaret.

## Analyzing Sales by Country

In [7]:
%%sql

WITH
one_customers AS
(
    select COUNT(customer_id) nr_customers, country from customer
    group by 2
    HAVING nr_customers = 1
),
new_customer AS
(
    SELECT *,
       CASE
        WHEN (country IN (SELECT country from one_customers)) THEN 'Other'
        ELSE country
        END AS new_country
        FROM customer
),
new_customer_sorted AS
(
    SELECT *, 
        CASE
            WHEN new_country = "Other" THEN 1
            ELSE 0
        END AS sort from new_customer
)

SELECT ncs.new_country country,
       COUNT(DISTINCT ncs.customer_id) Total_customers,
       ROUND(SUM(i.total), 2) Total_sales,
       ROUND(SUM(i.total) / COUNT(DISTINCT ncs.customer_id), 2) avg_sales_per_customer,
       ROUND(SUM(i.total) /  COUNT(i.invoice_id), 2) avg_order_value
    
FROM new_customer_sorted AS ncs
INNER JOIN invoice i ON i.customer_id=ncs.customer_id
GROUP BY 1
ORDER BY ncs.sort ASC







 * sqlite:///chinook.db
Done.


country,Total_customers,Total_sales,avg_sales_per_customer,avg_order_value
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77
Other,15,1094.94,73.0,7.45


In [17]:
%%sql
WITH
joined AS
(
SELECT il.*, t.* FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
)



SELECT invoice_id FROM invoice_line
EXCEPT
SELECT album_id FROM track



 * sqlite:///chinook.db
Done.


invoice_id
348
349
350
351
352
353
354
355
356
357


In [21]:
%%sql 
WITH
joined AS
(
SELECT il.*, t.* FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
)


SELECT invoice_id,
        MIN(track_id) first_track_id
FROM joined
GROUP BY 1

 * sqlite:///chinook.db
Done.


invoice_id,first_track_id
1,1158
2,201
3,2516
4,748
5,1986
6,30
7,42
8,81
9,196
10,2663
